In [1]:
import quandl, math
import numpy as np
import pandas as pd
from sklearn import preprocessing, model_selection, svm
from sklearn.linear_model import LinearRegression

In [2]:
df = quandl.get('WIKI/MSFT')
df.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
1986-03-13,25.50,29.25,25.5,28.00,3582600.0,0.0,1.0,0.059890,0.068697,0.059890,0.065761,1.031789e+09
1986-03-14,28.00,29.50,28.0,29.00,1070000.0,0.0,1.0,0.065761,0.069284,0.065761,0.068110,3.081600e+08
1986-03-17,29.00,29.75,29.0,29.50,462400.0,0.0,1.0,0.068110,0.069872,0.068110,0.069284,1.331712e+08
1986-03-18,29.50,29.75,28.5,28.75,235300.0,0.0,1.0,0.069284,0.069872,0.066936,0.067523,6.776640e+07
1986-03-19,28.75,29.00,28.0,28.25,166300.0,0.0,1.0,0.067523,0.068110,0.065761,0.066349,4.789440e+07


In [4]:
df = df[['Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume']]
df['HL_PCT'] = (df['Adj. High'] - df['Adj. Low'])/df['Adj. Close'] * 100.00
df['PCT_change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100.00
df = df[['Adj. Close', 'HL_PCT', 'PCT_change', 'Adj. Volume']]

In [5]:
forecast_col = 'Adj. Close'
df.fillna(value=-99999, inplace=True)
forecast_out=int(math.ceil(0.01 * len(df)))
df['label'] = df[forecast_col].shift(-forecast_out)
df.dropna(inplace=True)

In [7]:
X = np.array(df.drop(['label'], 1))
Y = np.array(df['label'])

X = preprocessing.scale(X)
Y = np.array(df['label'])

X_train, X_test, y_train, y_test = model_selection.train_test_split(X,Y,test_size=0.2)

In [9]:
clf = svm.SVR()
clf.fit(X_train, y_train)
confidence = clf.score(X_test, y_test)
confidence

0.94618286199134827

In [10]:
clf = LinearRegression()
clf.fit(X_train, y_train)
confidence = clf.score(X_test, y_test)
confidence

/Users/nicolasfeller/work/src/github.com/nrkfeller/tensorflowtutorial/tftut/lib/python3.6/site-packages/scipy/linalg/basic.py:884: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


0.95579574213169161

In [11]:
for k in ['linear','poly','rbf','sigmoid']:
    clf = svm.SVR(kernel=k)
    clf.fit(X_train, y_train)
    confidence = clf.score(X_test, y_test)
    print(k,confidence)

linear 0.955542054296
poly 0.557015094555
rbf 0.946182861991
sigmoid -67.4607879228
